In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import qiskit
from qiskit import BasicAer
from qiskit.aqua.input import ClassificationInput
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.components.feature_maps import SecondOrderExpansion
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.components.multiclass_extensions import AllPairs
from qiskit.aqua.utils.dataset_helper import get_feature_dimension
from qiskit.aqua.algorithms.many_sample.qsvm._qsvm_estimator import _QSVM_Estimator

In [ ]:
"""
from qiskit import IBMQ
token = 'ADD_TOKEN_HERE'
#my_provider = IBMQ.get_provider()
#backend = my_provider.get_backend('ibmq_qasm_simulator')
"""

In [ ]:
"""
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_qasm_simulator')
"""

In [ ]:
new_info = pd.read_csv(r"C:\Users\Albert\Documents\NSBE\ForexBot\DemoFiles\USD_JPY_h1_1000.csv")
new_info.head()

In [ ]:
#data_1_Bid = pd.read_csv("USDJPY_Long_Bid.csv")
#data_1_Ask = pd.read_csv("USDJPY_Long_Ask.csv")
data = pd.DataFrame({"AskClose":new_info['askclose'], 
                         "BidClose":new_info['bidclose']})

In [ ]:
#Used to prep data for classifiers, returns dataframe then array that contains lags
def Prep(data, lag_num):
    data = data.dropna()
    df = pd.DataFrame(data[['AskClose', 'BidClose']].mean(axis=1), columns=['midclose'])
    #print(data[:5]/data[:5].shift(1))
    df['returns'] = np.log(df / df.shift(1))
    lags = lag_num
    cols = []
    for lag in range(1, lags + 1):
        col = 'lag_%s' % lag
        df[col] = df['returns'].shift(lag)
        cols.append(col)
    
    df = df.dropna()
    return df, cols;

In [ ]:
df_1, cols = Prep(data,5)
df_1 = np.sign(df_1)
df_1 = df_1.replace([-1], 2)
print(df_1[cols])
np.isin(2, df_1[cols])

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(df_1[cols][920:].to_numpy(), df_1['returns'][920:].to_numpy(), test_size=0.25)


"""
train_x = np.sign(train_x)
test_x = np.sign(test_x)
train_y = np.sign(train_y)
test_y = np.sign(test_y)"""


num_features = 3
training_size = 1660
test_size = 66640
feature_map = SecondOrderExpansion(feature_dimension=num_features, depth=1)

In [ ]:
params = {
            'problem': {'name': 'classification'},
            'algorithm': {
                'name': 'QSVM',
            },
            'multiclass_extension': {'name': 'OneAgainstRest'},
            'feature_map': {'name': 'SecondOrderExpansion', 'depth': 1}
}
 

training_dataset={'Sell':train_x[train_y==2],
                #'Nothing':train_x[train_y==0],
                'Buy':train_x[train_y==1]}
test_dataset={'Sell':test_x[test_y==2],
                      #  'Nothing':test_x[test_y==0],
                        'Buy':test_x[test_y==1]}
#total_arr = np.concatenate((test_dataset['Sell'],test_dataset['Nothing'],test_dataset['Buy']))
total_arr = np.concatenate((test_dataset['Sell'],test_dataset['Buy']))
alg_input = ClassificationInput(training_dataset, test_dataset, total_arr)

In [ ]:
aqua_globals.random_seed = 1024

backend = BasicAer.get_backend('qasm_simulator')
feature_map = SecondOrderExpansion(feature_dimension=get_feature_dimension(training_dataset),
                                   depth=1, entangler_map=[[0, 1]])
svm = QSVM(feature_map, training_dataset,test_dataset, total_arr,
          multiclass_extension=AllPairs(_QSVM_Estimator, [feature_map],
                                       ))
quantum_instance = QuantumInstance(backend, shots=1000,
                                    seed_simulator=aqua_globals.random_seed,
                                    seed_transpiler=aqua_globals.random_seed,
                                  skip_qobj_validation=True,
                                  wait = None)

#"""
result = svm.run(quantum_instance)
for k,v in result.items():
    print("'{}' : {}".format(k, v))
#"""

In [ ]:
#svm.train(train_x, train_y, quantum_instance)
get_feature_dimension(training_dataset)

In [ ]:
y_pred = svm.predict(test_x)
print(accuracy_score(test_y, y_pred ) * 100, '%')
print(y_pred)
print(test_x)
print(test_y)
print( df_1['returns'][980:])

In [ ]:
import pickle
pickle.dump(svm, open('Quantum_model', 'wb'))

In [ ]:
import pickle
loaded_model = pickle.load(open('C:\\Users\\Albert\\Documents\\NSBE\\ForexBot\\Quantum\\models\\Quantum_model_3', 'rb'))
y_pred = loaded_model.predict(test_x[:20])

In [ ]:
print(accuracy_score(test_y[:20], y_pred ) * 100, '%')

In [ ]:
#pred = svm.predict(test_x[len(test_x)-5:])

In [ ]:
#print(test_y[len(test_y)-5:])
#print(pred)

In [ ]:
#test_x == test_y

In [ ]:
#import os
#os.path.abspath(os.getcwd())

In [ ]:
#svm.save_model("C:\\Users\\Albert\\Documents\\NSBE\\Quantum_Model_85%")

In [ ]:
#np.isin(2, train_x)

In [ ]:
#test_y